### 동적 크롤링(Selenium)

In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

def create_driver():
    """Selenium ChromeDriver를 생성하는 함수"""
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

def scroll_to_bottom(driver, max_attempts=30):
    """페이지 끝까지 스크롤을 내리는 함수"""
    scroll_heights = set()
    attempts = 0
    
    while attempts < max_attempts:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height in scroll_heights:
            break
        scroll_heights.add(new_height)
        attempts += 1

def crawl_naver_blog(query):
    """네이버 블로그 검색 결과를 크롤링하여 CSV로 저장하는 함수"""
    # 검색어를 URL에 맞게 인코딩
    from urllib.parse import quote
    encoded_query = quote(query)

    url = f"https://search.naver.com/search.naver?sm=tab_hty.top&ssc=tab.blog.all&query={encoded_query}"

    driver = create_driver()
    driver.get(url)

    # 스크롤 두 번
    scroll_to_bottom(driver)
    time.sleep(2)
    scroll_to_bottom(driver)

    # 페이지 HTML 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # 블로그 글 정보 수집
    blogs = []
    for link in soup.select("a.title_link"):
        title = link.get_text(strip=True)
        href = link.get("href")
        blogs.append({"Title": title, "URL": href})

    # 데이터프레임 생성 및 저장
    df = pd.DataFrame(blogs)
    filename = "naver_blog_posts.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")

    print(f"{len(blogs)}개의 블로그 글이 '{filename}'에 저장되었습니다.")

    driver.quit()

### 정적 크롤링(requests + BeautifulSoup)

In [ ]:
import urllib.request
import urllib.parse
import os
import json
import pandas as pd
from bs4 import BeautifulSoup

def naver_developer(search, site, max_results=500):
    """네이버 API를 사용하여 데이터를 검색하고 'title'과 'link'만 Pandas DataFrame으로 반환"""
    
    encText = urllib.parse.quote(search)
    client_id = os.getenv("NAVER_CLIENT_ID")
    client_secret = os.getenv("NAVER_CLIENT_SECRET")
    
    if not client_id or not client_secret:
        raise ValueError("NAVER_CLIENT_ID 또는 NAVER_CLIENT_SECRET이 설정되지 않았습니다.")

    all_results = [] 
    display = 100 
    
    for start in range(1, max_results + 1, display):  
        url = f"https://openapi.naver.com/v1/search/{site}.json?query={encText}&display={display}&start={start}&sort=date"
        print(f"Fetching: {url}")  

        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", client_id)
        request.add_header("X-Naver-Client-Secret", client_secret)

        try:
            response = urllib.request.urlopen(request)
            rescode = response.getcode()

            if rescode == 200:
                response_body = response.read()
                response_data = response_body.decode('utf-8')
                response_dict = json.loads(response_data)
                
                items = response_dict.get("items", [])
                
                # title과 link만 저장
                filtered_items = [{"title": item["title"], "link": item["link"]} for item in items]
                all_results.extend(filtered_items)  

                if len(items) < display:  # 마지막 페이지면 종료
                    break
            else:
                print(f"Error Code: {rescode}")
                break

        except urllib.error.HTTPError as e:
            print(f"HTTP Error: {e.code} - {e.reason}")
            break
        except urllib.error.URLError as e:
            print(f"URL Error: {e.reason}")
            break

    # Pandas DataFrame으로 변환
    csv_data = pd.DataFrame(all_results)

    return csv_data

def parsing(df, site):
    """ `BeautifulSoup`을 사용하여 HTML 태그 제거 후 기존 데이터프레임에 반영하고 저장 """

    def clean_html(text):
        """HTML 태그 제거"""
        if pd.isna(text):  # NaN 값이 있는 경우 빈 문자열로 대체
            return ""
        soup = BeautifulSoup(text, "html.parser")
        return soup.get_text().strip()  # HTML 태그 제거 후 공백 제거

    # 'title'에서 HTML 태그 제거 (link는 HTML 태그 없음)
    df['title'] = df['title'].apply(clean_html)

    # CSV 저장 경로 설정
    save_dir = "../data/csv"
    csv_file_name = f"naver_{site}.csv"
    save_path = os.path.join(save_dir, csv_file_name)

    os.makedirs(save_dir, exist_ok=True)

    df.to_csv(save_path, index=False, encoding="utf-8-sig")

    print(f"데이터가 정제되어 저장되었습니다: {save_path}")
    
    return df 